In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys

from pandas.io.json import json_normalize
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os
os.getcwd()

'/home/mrk_ram108/Kaggle'

In [2]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [str(column)+"."+str(subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
#train = load_df(os.getcwd()+'/train.csv')
#test = load_df()

#### Saving CSV files

In [4]:
#train.to_csv('train_extracted.csv',encoding='utf8')
train = pd.read_csv('train_extracted.csv',encoding='utf8')

/home/mrk_ram108/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Emtpy values 

In [5]:
100 * float(train['totals.transactionRevenue'].isnull().sum())/len(train['totals.transactionRevenue'])

98.72572768529513

#### Analysis


In [6]:
train.drop([u'Unnamed: 0'], axis = 1, inplace = True)
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [7]:
train.visitNumber.describe()

count    903653.000000
mean          2.264897
std           9.283735
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         395.000000
Name: visitNumber, dtype: float64

#### Finding columns with just one value in it

In [8]:
### Selecting columns with just one unique value and those with NULL values greater than 50% of the dataset
drop_columns = [each for each in train.columns if len(train[each].unique()) <= 2 or train[each].isnull().sum() > 0.5 * train.shape[0]]
drop_columns = [each for each in drop_columns if each != 'totals.transactionRevenue']

#### Response variable

In [9]:
train['totals.transactionRevenue'] = pd.to_numeric(train['totals.transactionRevenue'])
train['totals.transactionRevenue'].describe()

count    1.151500e+04
mean     1.337448e+08
std      4.482852e+08
min      1.000000e+04
25%      2.493000e+07
50%      4.945000e+07
75%      1.076550e+08
max      2.312950e+10
Name: totals.transactionRevenue, dtype: float64

In [10]:
%whos

Variable           Type         Data/Info
-----------------------------------------
LinearRegression   ABCMeta      <class 'sklearn.linear_mo<...>l.base.LinearRegression'>
datetime           type         <type 'datetime.datetime'>
drop_columns       list         n=32
each               unicode      trafficSource.referralPath
gc                 module       <module 'gc' (built-in)>
json               module       <module 'json' from '/hom<...>on2.7/json/__init__.pyc'>
json_normalize     function     <function json_normalize at 0x7f598c7fcb90>
load_df            function     <function load_df at 0x7f59873ea500>
np                 module       <module 'numpy' from '/ho<...>ages/numpy/__init__.pyc'>
os                 module       <module 'os' from '/home/<...>a2/lib/python2.7/os.pyc'>
pd                 module       <module 'pandas' from '/h<...>ges/pandas/__init__.pyc'>
plt                module       <module 'matplotlib.pyplo<...>s/matplotlib/pyplot.pyc'>
preprocessing      module      

In [11]:
train_upd = train.drop(drop_columns, axis = 1)

In [12]:
for each in train_upd.columns:
    print(each)
    print(train_upd[each].value_counts())
    print("\n")

channelGrouping
Organic Search    381561
Social            226117
Direct            143026
Referral          104838
Paid Search        25326
Affiliates         16403
Display             6262
(Other)              120
Name: channelGrouping, dtype: int64


date
20161128    4807
20161115    4685
20161114    4466
20161130    4435
20161026    4375
20161129    4337
20161116    4334
20161004    4322
20161205    4265
20170426    4224
20161201    4200
20161027    4162
20161121    4143
20161117    4074
20161024    4063
20161110    4055
20161103    4014
20161102    3960
20161122    3942
20161108    3899
20161025    3842
20161107    3832
20161031    3827
20161123    3805
20161028    3791
20161005    3770
20161124    3770
20161125    3759
20161020    3755
20161109    3752
            ... 
20160813    1596
20170430    1594
20161001    1589
20161226    1586
20170115    1576
20170429    1566
20170611    1555
20170204    1549
20170423    1548
20170610    1545
20170604    1534
20170114    1526
20170205  

Name: device.deviceCategory, dtype: int64


device.operatingSystem
Windows          350072
Macintosh        253938
Android          123892
iOS              107665
Linux             35034
Chrome OS         26337
(not set)          4695
Windows Phone      1216
Samsung             280
BlackBerry          218
Nintendo Wii        100
Firefox OS           89
Xbox                 66
Nintendo WiiU        35
FreeBSD               9
Nokia                 2
OpenBSD               2
SunOS                 1
NTT DoCoMo            1
Nintendo 3DS          1
Name: device.operatingSystem, dtype: int64


geoNetwork.city
not available in demo dataset    508229
Mountain View                     40884
(not set)                         34262
New York                          26371
San Francisco                     20329
Sunnyvale                         13086
London                            12607
San Jose                          10295
Los Angeles                        8670
Bangkok                         

Northern America      390657
Southeast Asia         77800
Southern Asia          59321
Western Europe         59114
Northern Europe        58168
Eastern Asia           46919
Eastern Europe         45249
South America          41731
Western Asia           38443
Southern Europe        35780
Central America        15583
Australasia            14893
Northern Africa         7683
Western Africa          2573
Caribbean               2406
Southern Africa         2169
Eastern Africa          1927
(not set)               1468
Central Asia            1215
Middle Africa            393
Melanesia                 81
Micronesian Region        55
Polynesia                 25
Name: geoNetwork.subContinent, dtype: int64


totals.hits
1      446754
2      137952
3       70402
4       42444
5       30939
6       23918
7       19518
8       15484
9       12959
10      10640
11       9264
12       7879
13       6881
14       6194
15       5384
16       4716
17       4130
18       3755
19       3291
20       

In [13]:
## Cleaning the character columns in the data

for each in train_upd.columns:
    if(train_upd[each].dtype == 'O' and len(train_upd[each].unique()) < 500000):
        
        ### Skipping the values are actually numeric or float
        
        if( not (len(pd.to_numeric(train_upd[each],errors = 'coerce').unique()) < 0.5 * len(train_upd[each].unique())) ):
            train_upd[each] = train_upd[each].fillna(train_upd[each].mean())
        else:


            print(each)
            print(len(train_upd[each].unique()))
            train_upd[each] = train_upd[each].apply(lambda x: x.lower().strip())
            extreme_values = train_upd[each].value_counts()[train_upd[each].value_counts() < 10].index

            ## Replacing the outliers (extreme) values with 'Others'

            train_upd[each] = train_upd[each].apply(lambda x: np.where(x in extreme_values, 'Others',x))


channelGrouping
8
device.browser
54
device.deviceCategory
3
device.operatingSystem
20
geoNetwork.city
649
geoNetwork.continent
6
geoNetwork.country
222
geoNetwork.metro
94
geoNetwork.networkDomain
28064
geoNetwork.region
376
geoNetwork.subContinent
23
trafficSource.campaign
10
trafficSource.medium
7
trafficSource.source
380


#### Creating date variables

In [20]:
train_upd['year'] = train_upd.date.apply(lambda x: int(str(x)[0:4]))
train_upd['month'] = train_upd.date.apply(lambda x: int(str(x)[4:6]))
train_upd['day'] = train_upd.date.apply(lambda x: int(str(x)[6:8]))

In [22]:
## Removing 'geoNetwork.networkDomain' column for now, since it has a lot of categories - over 3000

#train_upd = train_upd.drop(['geoNetwork.networkDomain'], axis = 1)
train_upd.shape

(903653, 25)

#### Dividing the data into train and test

In [47]:
train_upd['totals.transactionRevenue'] = train_upd['totals.transactionRevenue'].fillna(0)
train_upd = train_upd.fillna(0)

In [48]:
train_upd.apply(lambda x: len(x.unique()))

channelGrouping                   8
date                            366
fullVisitorId                723803
sessionId                    902755
visitId                      886303
visitNumber                     384
visitStartTime               887159
device.browser                   27
device.deviceCategory             3
device.operatingSystem           15
geoNetwork.city                 482
geoNetwork.continent              6
geoNetwork.country              188
geoNetwork.metro                 79
geoNetwork.region               318
geoNetwork.subContinent          23
totals.hits                     274
totals.pageviews                214
totals.transactionRevenue      5333
trafficSource.campaign            9
trafficSource.medium              7
trafficSource.source            124
year                              2
month                            12
day                              31
dtype: int64

In [49]:
### Removing id columns
id_cols = ['sessionId','fullVisitorId','visitId']
train_dummies = pd.get_dummies(train_upd.drop(id_cols, axis = 1), dummy_na = True)

In [50]:
train_dummies.shape

(903653, 1311)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(train_dummies.drop(['totals.transactionRevenue'], axis = 1), \
                 train_dummies['totals.transactionRevenue'],test_size = 0.2, random_state = 9)

In [52]:
X_train.head()

,date,visitNumber,visitStartTime,totals.hits,totals.pageviews,year,month,day,channelGrouping_(other),channelGrouping_affiliates,...,trafficSource.source_tw.search.yahoo.com,trafficSource.source_uk.businessinsider.com,trafficSource.source_uk.search.yahoo.com,trafficSource.source_us.search.yahoo.com,trafficSource.source_vk.com,trafficSource.source_wap.sogou.com,trafficSource.source_wheretoget.it,trafficSource.source_yahoo,trafficSource.source_youtube.com,trafficSource.source_nan
40485,20170630,1,1498884588,4,4.0,2017,6,30,0,0,...,0,0,0,0,0,0,0,0,0,0
784765,20170727,1,1501200752,1,1.0,2017,7,27,0,0,...,0,0,0,0,0,0,0,0,0,0
103499,20170704,1,1499169217,1,1.0,2017,7,4,0,0,...,0,0,0,0,0,0,0,0,0,0
363033,20161223,4,1482550584,1,1.0,2016,12,23,0,0,...,0,0,0,0,0,0,0,0,0,0
886064,20170717,5,1500310627,3,2.0,2017,7,17,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
y_train.shape

(722922,)

In [54]:
lr_model = LinearRegression(n_jobs = -1)

In [65]:
#lr_model.fit(X_train.iloc[:,:250],y_train)

In [45]:
X_train.iloc[:,4].isnull().sum()

86